# Importing Libraries

In [ ]:
import pandas as pd
import networkx as nx
import torch
import sentence_transformers

# Importing Dataset

In [3]:
actors = pd.read_csv('../Dataset/name.basics.tsv', sep='\t', nrows=500)
titles = pd.read_csv('../Dataset/title.basics.tsv', sep='\t', nrows=500)
crew = pd.read_csv('../Dataset/title.crew.tsv', sep='\t', nrows=500)
episodes = pd.read_csv('../Dataset/title.episode.tsv', sep='\t', nrows=500)
principals = pd.read_csv('../Dataset/title.principals.tsv', sep='\t', nrows=500)
ratings = pd.read_csv('../Dataset/title.ratings.tsv', sep='\t', nrows=500)

In [ ]:
def load_node_csv(path, index_col, encoders=None, **kwargs):
    df = pd.read_csv(path, index_col=index_col, **kwargs)
    mapping = {index: i for i, index in enumerate(df.index.unique())}

    x = None
    if encoders is not None:
        xs = [encoder(df[col]) for col, encoder in encoders.items()]
        x = torch.cat(xs, dim=-1)

    return x, mapping

In [ ]:
G = nx.graph()


,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"actor,miscellaneous,producer","tt0050419,tt0072308,tt0053137,tt0027125"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack,archive_footage","tt0037382,tt0075213,tt0117057,tt0038355"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,music_department,producer","tt0057345,tt0049189,tt0056404,tt0054452"
3,nm0000004,John Belushi,1949,1982,"actor,writer,music_department","tt0072562,tt0077975,tt0080455,tt0078723"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0050986,tt0069467,tt0083922,tt0050976"
...,...,...,...,...,...,...
495,nm0000496,Juliette Lewis,1973,\N,"actress,director,producer","tt1322269,tt0101540,tt0110632,tt0116367"
496,nm0000497,Jennifer Lien,1974,\N,"actress,producer,script_department","tt0112178,tt0120586,tt0133189,tt0106100"
497,nm0000498,Matthew Lillard,1970,\N,"actor,producer,director","tt0267913,tt0133189,tt1033575,tt0331632"
498,nm0000499,Bai Ling,\N,\N,"actress,director,producer","tt0119994,tt0109506,tt1121931,tt0405336"
